In [1]:
import pandas as pd
import  numpy as np

In [2]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer

In [3]:
df = pd.read_csv('titanic.csv',usecols =['Age','Fare','Survived'])

In [4]:
df.head()

,Survived,Age,Fare
0,0,34.5,7.8292
1,1,47.0,7.0000
2,0,62.0,9.6875
3,0,27.0,8.6625
4,1,22.0,12.2875


In [5]:
df.dropna(inplace = True)

In [6]:
df.shape

(331, 3)

In [7]:
df.head()

,Survived,Age,Fare
0,0,34.5,7.8292
1,1,47.0,7.0000
2,0,62.0,9.6875
3,0,27.0,8.6625
4,1,22.0,12.2875


In [37]:
x = df.iloc[:,1:]
X =df.iloc[:,1:]
y =df.iloc[:,0]

In [9]:
x_train,x_test , y_train, y_test = train_test_split(x,y,test_size=0.2 , random_state=1)

In [10]:
x_train.head(2)

,Age,Fare
375,45.0,262.375
204,25.0,10.500


In [11]:
clf = DecisionTreeClassifier()

In [12]:
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [13]:
accuracy_score(y_test ,y_pred)

0.44776119402985076

In [14]:
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.5559714795008912

In [15]:
kbin_age = KBinsDiscretizer(n_bins=15,encode='ordinal',strategy ='quantile')
kbin_fare = KBinsDiscretizer(n_bins = 15,encode ='ordinal',strategy ='quantile')

In [16]:
trf = ColumnTransformer([
    ('fi',kbin_age,[0]),
    ('se',kbin_fare,[1])
])

In [17]:
x_train_trf = trf.fit_transform(x_train)
x_test_trf = trf.transform(x_test)

In [18]:
trf.named_transformers_['fi'].bin_edges_

array([array([ 0.17      ,  9.        , 18.        , 20.6       , 22.        ,
              23.        , 25.        , 26.        , 28.        , 30.        ,
              33.        , 37.        , 42.4       , 47.        , 54.46666667,
              76.        ])                                                   ],
      dtype=object)

In [19]:
output = pd.DataFrame({
    'age': x_train['Age'],
    'age_trf': x_train_trf[:,0],
    'fare':x_train['Fare'],
    'fare_trf': x_train_trf[:,1]
})

In [20]:
output['age_labels'] = pd.cut(x=x_train['Age'],
                                    bins=trf.named_transformers_['fi'].bin_edges_[0].tolist())
output['fare_labels'] = pd.cut(x=x_train['Fare'],
                                    bins=trf.named_transformers_['se'].bin_edges_[0].tolist())
output.sample(5)

,age,age_trf,fare,fare_trf,age_labels,fare_labels
90,22.0,4.0,13.9000,6.0,"(20.6, 22.0]","(13.0, 14.5]"
239,48.0,13.0,106.4250,13.0,"(47.0, 54.467]","(78.679, 151.55]"
361,24.0,5.0,37.0042,10.0,"(23.0, 25.0]","(29.042, 39.0]"
381,26.0,7.0,7.8792,2.0,"(25.0, 26.0]","(7.796, 7.913]"
194,8.0,0.0,32.5000,10.0,"(0.17, 9.0]","(29.042, 39.0]"


In [21]:
clf = DecisionTreeClassifier()
clf.fit(x_train_trf,y_train)
y_pred2 = clf.predict(x_test_trf)

In [22]:
accuracy_score(y_test,y_pred2)

0.5373134328358209

In [23]:
x_trf = trf.fit_transform(x)
np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy'))

0.5649732620320854

In [40]:
def dist(bins,strategy):
    kbin_age = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    kbin_fare = KBinsDiscretizer(n_bins=bins,encode='ordinal',strategy=strategy)
    
    trf = ColumnTransformer([
        ('first',kbin_age,[0]),
        ('second',kbin_fare,[1])
    ])
    
    x_trf = trf.fit_transform(x)
    print(np.mean(cross_val_score(DecisionTreeClassifier(),x,y,cv=10,scoring='accuracy')))
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(x['Age'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(x_trf[:,0],color='red')
    plt.title("After")

    plt.show()
    
    plt.figure(figsize=(14,4))
    plt.subplot(121)
    plt.hist(x['Fare'])
    plt.title("Before")

    plt.subplot(122)
    plt.hist(x_trf[:,1],color='red')
    plt.title("Fare")

    plt.show()

In [42]:
dist(5,'kmeans')

AttributeError: 'NoneType' object has no attribute 'split'